In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/10000-amazon-products-dataset/Amazon_Products.csv


```
1) Pick a dataset

2) Pose at least three questions related to business or real-world applications of how the data could be used.

3) Create a Jupyter Notebook, using any associated packages you'd like, to:

    a. Prepare data:
        - Gather necessary data to answer your questions
        - Handle categorical and missing data
        - Provide insight into the methods you chose and why you chose them
    b. Analyze, Model, and Visualize
        - Provide a clear connection between your business questions and how the data answers them
4) Communicate your business insights:
    - Create a Github repository to share your code and data wrangling/modeling techniques, with a technical audience in mind
    - Create a blog post to share your questions and insights with a non-technical audience
```

In [2]:
df = pd.read_csv("/kaggle/input/10000-amazon-products-dataset/Amazon_Products.csv", dtype = str)
df.head()

,uniq_id,product_name,manufacturer,price,number_available_in_stock,number_of_reviews,number_of_answered_questions,average_review_rating,amazon_category_and_sub_category,description,...,Unnamed: 885,Unnamed: 886,Unnamed: 887,Unnamed: 888,Unnamed: 889,Unnamed: 890,Unnamed: 891,Unnamed: 892,Unnamed: 893,Unnamed: 894
0,eac7efa5dbd3d667f26eb3d3ab504464,Hornby 2014 Catalogue,Hornby,£3.42,5 new,15,1,4.9 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,Product Description Hornby 2014 Catalogue Box ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,b17540ef7e86e461d37f3ae58b7b72ac,FunkyBuys® Large Christmas Holiday Express Fes...,FunkyBuys,£16.99,NaN,2,1,4.5 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,Size Name:Large FunkyBuys® Large Christmas Hol...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,348f344247b0c1a935b1223072ef9d8a,CLASSIC TOY TRAIN SET TRACK CARRIAGES LIGHT EN...,ccf,£9.99,2 new,17,2,3.9 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,e12b92dbb8eaee78b22965d2a9bbbd9f,HORNBY Coach R4410A BR Hawksworth Corridor 3rd,Hornby,£39.99,NaN,1,2,5.0 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,Hornby 00 Gauge BR Hawksworth 3rd Class W 2107...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,e33a9adeed5f36840ccc227db4682a36,Hornby 00 Gauge 0-4-0 Gildenlow Salt Co. Steam...,Hornby,£32.19,NaN,3,2,4.7 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,Product Description Hornby RailRoad 0-4-0 Gild...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# import sqlite3
# connection = sqlite3.connect("/kaggle/input/10000-amazon-products-dataset/Amazon_Products.csv")
# connection

## 1. Evaluate each column and covert it into a usable format

In [4]:
# delete columns which are unnamed
unnamed_cols = [col for col in df.columns if 'Unnamed' in col]
# print("Need to delete these cols: ", unnamed_cols)
df = df.drop(unnamed_cols, axis = 1)
print("Leftover columns: ", df.columns)
df.shape

Leftover columns:  Index(['uniq_id', 'product_name', 'manufacturer', 'price',
       'number_available_in_stock', 'number_of_reviews',
       'number_of_answered_questions', 'average_review_rating',
       'amazon_category_and_sub_category', 'description',
       'product_information', 'product_description',
       'items_customers_buy_after_viewing_this_item',
       'customer_questions_and_answers'],
      dtype='object')


(10004, 14)

In [5]:
def ColSummary (col, dtype=None, new_col=None):
    try:
        print("\n"+str(col).upper()+" ANALYSIS")
        if new_col:
            df.rename(columns={col:new_col})
            print("\t"+col+" name updated to "+new_col)
            col = new_col
        if dtype:
            df[col] = df[col].astype(dtype)
            print("\tType changed to "+dtype)
        print('\tTotal number of values : ', len(df[col]))
        print('\tTotal number of unique values : ', df[col].nunique())
        print('\tUnique values : ',df[col].unique()[:5])
        print('\tRange of length : ', min(df[col].apply(lambda x: len(str(x))))," to ",max(df[col].apply(lambda x: len(str(x)))))
        print('\tMissing Values : ', df[col].isna().sum())
    except:
        print("\n*** Cannot work with this datatype.. Try Again.... ***")

In [6]:
# remove rows which have non-sensical ids
ColSummary('uniq_id', 'str')
print('\n\tInspecting IDs with unusual length : ', list(df['uniq_id'][df['uniq_id'].apply(lambda x: len(str(x)))<=3]))
print('\tDeleting IDs with unusual length ..... ')
df = df.drop(index = list(df['uniq_id'][df['uniq_id'].apply(lambda x: len(str(x))<=3)].index))
print('\nUnique IDs updated .....')
ColSummary('uniq_id')


UNIQ_ID ANALYSIS
	Type changed to str
	Total number of values :  10004
	Total number of unique values :  10002
	Unique values :  ['eac7efa5dbd3d667f26eb3d3ab504464' 'b17540ef7e86e461d37f3ae58b7b72ac'
 '348f344247b0c1a935b1223072ef9d8a' 'e12b92dbb8eaee78b22965d2a9bbbd9f'
 'e33a9adeed5f36840ccc227db4682a36']
	Range of length :  1  to  32
	Missing Values :  0

	Inspecting IDs with unusual length :  ['nan', '}', 'nan', '}']
	Deleting IDs with unusual length ..... 

Unique IDs updated .....

UNIQ_ID ANALYSIS
	Total number of values :  10000
	Total number of unique values :  10000
	Unique values :  ['eac7efa5dbd3d667f26eb3d3ab504464' 'b17540ef7e86e461d37f3ae58b7b72ac'
 '348f344247b0c1a935b1223072ef9d8a' 'e12b92dbb8eaee78b22965d2a9bbbd9f'
 'e33a9adeed5f36840ccc227db4682a36']
	Range of length :  32  to  32
	Missing Values :  0


In [7]:
ColSummary('product_name', 'string')
ColSummary('manufacturer', 'string')


PRODUCT_NAME ANALYSIS
	Type changed to string
	Total number of values :  10000
	Total number of unique values :  9964
	Unique values :  <StringArray>
[                                                                                                 'Hornby 2014 Catalogue',
 'FunkyBuys® Large Christmas Holiday Express Festive Train Set (SI-TY1017) Toy Light / Sounds / Battery Operated & Smoke',
                                             'CLASSIC TOY TRAIN SET TRACK CARRIAGES LIGHT ENGINE BOXED BOYS KIDS BATTERY',
                                                                         'HORNBY Coach R4410A BR Hawksworth Corridor 3rd',
                                                        'Hornby 00 Gauge 0-4-0 Gildenlow Salt Co. Steam Locomotive Model']
Length: 5, dtype: string
	Range of length :  3  to  536
	Missing Values :  0

MANUFACTURER ANALYSIS
	Type changed to string
	Total number of values :  10000
	Total number of unique values :  2651
	Unique values :  <StringArray>
['Horn

In [8]:
# convert currency into float
ColSummary('price')
print("\n Deleting rows which have price with greater than 5 length i.e. have ranges instead of values .....")
df = df.drop(index = list(df['price'][df['price'].apply(lambda x: len(str(x))>5)].index))
df['price'] = df['price'][df['price'].isna()==False].str[1:]
ColSummary('price', 'float64')


PRICE ANALYSIS
	Total number of values :  10000
	Total number of unique values :  2625
	Unique values :  ['£3.42' '£16.99' '£9.99' '£39.99' '£32.19']
	Range of length :  3  to  19
	Missing Values :  1435

 Deleting rows which have price with greater than 5 length i.e. have ranges instead of values .....

PRICE ANALYSIS
	Type changed to float64
	Total number of values :  5603
	Total number of unique values :  754
	Unique values :  [3.42 9.99 6.99  nan 9.6 ]
	Range of length :  3  to  4
	Missing Values :  1435


In [9]:
# change to string by removing extra text
ColSummary('number_available_in_stock')
df['number_available_in_stock'] = df['number_available_in_stock'].apply(lambda x: str(x).split()[0])
ColSummary('number_available_in_stock','float64')


NUMBER_AVAILABLE_IN_STOCK ANALYSIS
	Total number of values :  5603
	Total number of unique values :  70
	Unique values :  ['5\xa0new' '2\xa0new' nan '1\xa0new' '18\xa0new']
	Range of length :  3  to  13
	Missing Values :  1532

NUMBER_AVAILABLE_IN_STOCK ANALYSIS
	Type changed to float64
	Total number of values :  5603
	Total number of unique values :  55
	Unique values :  [ 5.  2. nan  1. 18.]
	Range of length :  3  to  4
	Missing Values :  1532


In [10]:
ColSummary('number_of_reviews')
print("\n Removing commas from strings for float conversion .....")
df['number_of_reviews'] = df['number_of_reviews'].apply(lambda x: str(x).replace(',',''))
ColSummary('number_of_reviews', 'float64')
ColSummary('number_of_answered_questions','float64')


NUMBER_OF_REVIEWS ANALYSIS
	Total number of values :  5603
	Total number of unique values :  157
	Unique values :  ['15' '17' '2' '8' '1']
	Range of length :  1  to  5
	Missing Values :  7

 Removing commas from strings for float conversion .....

NUMBER_OF_REVIEWS ANALYSIS
	Type changed to float64
	Total number of values :  5603
	Total number of unique values :  157
	Unique values :  [15. 17.  2.  8.  1.]
	Range of length :  3  to  6
	Missing Values :  7

NUMBER_OF_ANSWERED_QUESTIONS ANALYSIS
	Type changed to float64
	Total number of values :  5603
	Total number of unique values :  17
	Unique values :  [ 1.  2.  6. nan  5.]
	Range of length :  3  to  4
	Missing Values :  528


In [11]:
ColSummary('average_review_rating')
df['average_review_rating'] = df['average_review_rating'].apply(lambda x: str(x).split()[0])
ColSummary('average_review_rating')


AVERAGE_REVIEW_RATING ANALYSIS
	Total number of values :  5603
	Total number of unique values :  17
	Unique values :  ['4.9 out of 5 stars' '3.9 out of 5 stars' '5.0 out of 5 stars'
 '4.8 out of 5 stars' '4.5 out of 5 stars']
	Range of length :  3  to  18
	Missing Values :  7

AVERAGE_REVIEW_RATING ANALYSIS
	Total number of values :  5603
	Total number of unique values :  18
	Unique values :  ['4.9' '3.9' '5.0' '4.8' '4.5']
	Range of length :  3  to  3
	Missing Values :  0


In [12]:
ColSummary('amazon_category_and_sub_category')
df['amazon_category_and_sub_category'] = df['amazon_category_and_sub_category'].apply(lambda x: str(x).split(" > "))
itr = max(df['amazon_category_and_sub_category'].apply(lambda x: len(x)))
for i in range (1, itr+1):
    df['amazon_category_and_sub_category'].apply(lambda x: x.append("None") if (len(x) < i) else x)
                                                                                      
ColSummary('amazon_category_and_sub_category')


AMAZON_CATEGORY_AND_SUB_CATEGORY ANALYSIS
	Total number of values :  5603
	Total number of unique values :  220
	Unique values :  ['Hobbies > Model Trains & Railway Sets > Rail Vehicles > Trains'
 'Hobbies > Model Trains & Railway Sets > Lighting & Signal Engineering > Lamps & Lighting'
 'Hobbies > Model Trains & Railway Sets > Rail Vehicles > Locomotives'
 'Hobbies > Model Trains & Railway Sets > Lighting & Signal Engineering > Signal & Sound'
 'Hobbies > Model Trains & Railway Sets > Accessories > Recreational Activities']
	Range of length :  3  to  120
	Missing Values :  330

AMAZON_CATEGORY_AND_SUB_CATEGORY ANALYSIS
	Total number of values :  5603

*** Cannot work with this datatype.. Try Again.... ***


In [13]:
ColSummary('description')
ColSummary('product_information')
ColSummary('product_description')
ColSummary('items_customers_buy_after_viewing_this_item')
ColSummary('customer_questions_and_answers')


DESCRIPTION ANALYSIS
	Total number of values :  5603
	Total number of unique values :  4646
	Unique values :  ['Product Description Hornby 2014 Catalogue Box Contains 1 x one catalogue'
 'BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT ENGINE SOUND BOXED KIDS BATTERY Railway Train Set with Light Sound, Big Size Curved Track Free Wheeling Action Working Headlight Sound Horn The Track Can Also Be Assembled In More Layouts Length Of Track 104 cm WIDTH OF THE TRACK 68 CM Finely Detailed Realistic Toy Train REQUIRES 2 AA BATTERIES (NOT INCLUDED)'
 'These delicate model garden lights are mainly used in teaching, photography, and various kinds of scene model. Each of them is completed with wires and has double heads, which can be lighten with 6V power. These model lights measure about 7cm high. Light up your model layout with this great model lamppost! Description: A pack of 20pcs model garden lamps Each has double head, completed with wires and bulbs Mainly used to decorate your model layout

## 2. Handle Missing Data

## Question 1. 

In [14]:
str.replace?

In [15]:
df['amazon_category_and_sub_category'].apply(lambda x: x[4])

0       None
2       None
5       None
9       None
12      None
        ... 
9987    None
9988    None
9995    None
9997    None
9998    None
Name: amazon_category_and_sub_category, Length: 5603, dtype: object

## Question 2. 

## Question 3. Can we predict the amazon categories using description?